In [1]:
import numpy as np
import cv2 as cv
import pytesseract
from pytesseract import Output
import re
import math
import time
import csv
import pandas as pd
import os
import urllib.request
from flask import Flask, render_template, Response,jsonify
app = Flask(__name__)

In [2]:
net = cv.dnn.readNet("yolov3.weights","yolov3.cfg")
classes = []
with open("yolov3.txt","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [3]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [4]:
colors= np.random.uniform(0,255,size=(len(classes),3))
l=['banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake']

In [5]:
a=pd.read_csv('nutri.csv',index_col='name')

In [6]:
def gen_frames():
    cap=cv.VideoCapture(0) #0 for 1st webcam
    font = cv.FONT_HERSHEY_PLAIN
    starting_time= time.time()
    while True:
        _,frame= cap.read() # 
        #frame_id+=1
    
        height,width,channels = frame.shape
        #detecting objects
        blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320    

        
        net.setInput(blob)
        outs = net.forward(outputlayers)
        #print(outs[1])


        #Showing info on screen/ get confidence score of algorithm in detecting an object in blob
        class_ids=[]
        confidences=[]
        boxes=[]
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.3:
                    #onject detected
                    center_x= int(detection[0]*width)
                    center_y= int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)

                    #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                    #rectangle co-ordinaters
                    x=int(center_x - w/2)
                    y=int(center_y - h/2)
                    #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                    boxes.append([x,y,w,h]) #put all rectangle areas
                    confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                    class_ids.append(class_id) #name of the object tha was detected

        indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)


        for i in range(len(boxes)):
            if i in indexes:
                x,y,w,h = boxes[i]
                p=math.sqrt(w**2+h**2)
                
                label = str(classes[class_ids[i]])
                l=['banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake']
                brbr=['orange']
                if label in l:
                    confidence= confidences[i]
                    color = colors[class_ids[i]]
                    cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
                    cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
                    if label in brbr:
                        cv2.putText(frame,"Calories (kcal) "+str(a.loc[label,"Calories (kcal)"]),(x,y+100),font,1,(255,255,255),2)
                        cv2.putText(frame,"Total Sugar (g) "+str(a.loc[label,"Total Sugar (g)"]),(x,y+150),font,1,(255,255,255),2)
                        cv2.putText(frame,"Protein (g) "+str(a.loc[label,"Protein (g)"]),(x,y+200),font,1,(255,255,255),2)
            

        
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

In [7]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/static/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/static/video.html',methods=['GET', 'POST'])
def index_social():
    if request.method == 'GET':
        return redirect(url_for('video_feed'))
    """Video streaming home page."""
    return render_template('video.html')

@app.route('/static/index.html',methods=['GET', 'POST'])
def index_home():
       return render_template('index.html')
    
@app.route('/static/dosha-quiz.html',methods=['GET', 'POST'])
def dosha_quiz():
       return render_template('dosha-quiz.html')


In [8]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Feb/2022 20:10:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 20:10:42] "GET /about-image.jpg HTTP/1.1" 404 -
127.0.0.1 - - [18/Feb/2022 20:10:42] "GET /image/banner-bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [18/Feb/2022 20:19:45] "GET /static/dosha-quiz.html HTTP/1.1" 200 -
